In [ ]:
from PIL import Image
import os

# --- Helper Functions for Bit Manipulation and Text Conversion ---

def text_to_binary(text):
    """Converts a string of text into its binary representation."""
    # Convert each character to its 8-bit ASCII/Unicode representation, then join
    return ''.join(format(ord(char), '08b') for char in text)

def binary_to_text(binary_string):
    """Converts a binary string back into text."""
    text = ""
    # Iterate through the binary string 8 bits at a time
    for i in range(0, len(binary_string), 8):
        byte = binary_string[i:i+8]
        if len(byte) == 8: # Ensure it's a full byte
            text += chr(int(byte, 2))
    return text

def hide_bit_in_pixel(pixel_value, bit):
    """
    Modifies the least significant bit (LSB) of a pixel's color component.
    pixel_value: an integer (0-255) representing R, G, or B.
    bit: '0' or '1' (string).
    """
    # Clear the LSB (make it 0)
    new_pixel_value = pixel_value & 0xFE # Equivalent to pixel_value & 254
    # Set the LSB to the given bit
    if bit == '1':
        new_pixel_value |= 0x01 # Equivalent to new_pixel_value | 1
    return new_pixel_value

def extract_bit_from_pixel(pixel_value):
    """Extracts the least significant bit (LSB) from a pixel's color component."""
    return str(pixel_value & 0x01) # Equivalent to pixel_value & 1

# --- Main Steganography Functions ---

def embed_message(image_path, message, output_path="steg_image.jpg"):
    """
    Hides a text message within a JPG image using LSB steganography.

    Args:
        image_path (str): Path to the cover JNG image.
        message (str): The text message to hide.
        output_path (str): Path to save the steganographic image.
    """
    try:
        img = Image.open(image_path).convert("RGB") # Ensure image is in RGB format
    except FileNotFoundError:
        print(f"Error: Image file not found at '{image_path}'")
        return False
    except Exception as e:
        print(f"Error opening image: {e}")
        return False

    width, height = img.size
    pixels = img.load() # Load pixel data for direct manipulation

    # Append a delimiter to the message to know where it ends
    # This delimiter is crucial for extraction. Using a unique sequence.
    delimiter = "#END#"
    full_message = message + delimiter
    binary_message = text_to_binary(full_message)
    message_length_bits = len(binary_message)

    # Calculate required pixels: 3 bits per pixel (R, G, B)
    # We need 1 pixel for every 3 bits (approx) for 3 channels.
    # Each character is 8 bits. We use 3 bits per pixel for R, G, B channels.
    # So, 3 pixels (9 bits) can store just over 1 character.
    # Total pixels needed = ceil(message_length_bits / 3)
    required_pixels = (message_length_bits + 2) // 3 # Integer division, equivalent to ceil

    if required_pixels > width * height:
        print(f"Error: Image is too small to hide the message.")
        print(f"Message requires {required_pixels} pixels, but image only has {width * height} pixels.")
        return False

    message_index = 0
    for y in range(height):
        for x in range(width):
            if message_index < message_length_bits:
                r, g, b = pixels[x, y]

                # Modify R channel LSB
                if message_index < message_length_bits:
                    r = hide_bit_in_pixel(r, binary_message[message_index])
                    message_index += 1
                
                # Modify G channel LSB
                if message_index < message_length_bits:
                    g = hide_bit_in_pixel(g, binary_message[message_index])
                    message_index += 1
                
                # Modify B channel LSB
                if message_index < message_length_bits:
                    b = hide_bit_in_pixel(b, binary_message[message_index])
                    message_index += 1
                
                pixels[x, y] = (r, g, b)
            else:
                break # All message bits embedded
        if message_index >= message_length_bits:
            break

    try:
        img.save(output_path)
        print(f"Message successfully embedded into '{output_path}'")
        print(f"Original image size: {os.path.getsize(image_path)} bytes")
        print(f"Stego image size: {os.path.getsize(output_path)} bytes")
        return True
    except Exception as e:
        print(f"Error saving image: {e}")
        return False

def extract_message(steg_image_path):
    """
    Extracts a hidden text message from a LSB-stegged JNG image.

    Args:
        steg_image_path (str): Path to the steganographic image.

    Returns:
        str: The extracted hidden message, or None if an error occurs.
    """
    try:
        img = Image.open(steg_image_path).convert("RGB")
    except FileNotFoundError:
        print(f"Error: Stego image file not found at '{steg_image_path}'")
        return None
    except Exception as e:
        print(f"Error opening image: {e}")
        return None

    width, height = img.size
    pixels = img.load()

    binary_message = ""
    delimiter = "#END#"
    delimiter_binary = text_to_binary(delimiter)
    
    # We'll extract bits until we find the delimiter
    max_bits_to_extract = width * height * 3 # Max possible bits

    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            
            binary_message += extract_bit_from_pixel(r)
            if delimiter_binary in binary_message: break
            
            binary_message += extract_bit_from_pixel(g)
            if delimiter_binary in binary_message: break
            
            binary_message += extract_bit_from_pixel(b)
            if delimiter_binary in binary_message: break
            
        if delimiter_binary in binary_message:
            break

    # Find the delimiter and extract the actual message
    try:
        delimiter_index = binary_message.index(delimiter_binary)
        hidden_binary = binary_message[:delimiter_index]
        extracted_message = binary_to_text(hidden_binary)
        print(f"Message successfully extracted from '{steg_image_path}'")
        return extracted_message
    except ValueError:
        print(f"Error: Delimiter '{delimiter}' not found in extracted data. No hidden message or image is not stego'd.")
        return None
    except Exception as e:
        print(f"Error during message extraction: {e}")
        return None

# --- Main Program Execution ---

def main():
    print("--- Image LSB Steganography Tool (ImageStegHide) ---")
    print("1. Embed a message into an image")
    print("2. Extract a message from an image")
    print("3. Exit")

    while True:
        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            input_image_path = input("Enter path to the cover JPG image (e.g., my_image.jpg): ")
            message_to_hide = input("Enter the text message to hide: ")
            output_image_path = input("Enter output path for the stego image (e.g., stego_image.jpg): ")
            
            if not input_image_path.lower().endswith(".jpg"):
                print("Warning: It's highly recommended to use JPG images for LSB steganography as they are lossless.")
                # You might add a check here to convert if not PNG, or strictly enforce PNG

            if embed_message(input_image_path, message_to_hide, output_image_path):
                print("Embedding complete.")
            else:
                print("Embedding failed.")

        elif choice == '2':
            steg_image_path = input("Enter path to the steganographic JPG image: ")
            
            if not steg_image_path.lower().endswith(".jpg"):
                print("Warning: This tool is designed for JPG images. Results may be unpredictable with other formats.")

            extracted_msg = extract_message(steg_image_path)
            if extracted_msg is not None:
                print(f"\nExtracted Message:\n---\n{extracted_msg}\n---")
            else:
                print("Extraction failed or no message found.")

        elif choice == '3':
            print("Exiting ImageStegHide. Goodbye!")
            break1
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")
if __name__ == "__main__":
    main()

--- Image LSB Steganography Tool (ImageStegHide) ---
1. Embed a message into an image
2. Extract a message from an image
3. Exit


Enter your choice (1/2/3):  1
Enter path to the cover JPG image (e.g., my_image.jpg):  C:\Users\rimar\OneDrive\Desktop
Enter the text message to hide:  ashish is a good boy and intelligent
Enter output path for the stego image (e.g., stego_image.jpg):  v


Error opening image: [Errno 13] Permission denied: 'C:\\Users\\rimar\\OneDrive\\Desktop'
Embedding failed.
